In [52]:
import pandas as pd
import csv
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
#print(nltk.__version__) #trying to remember if i installed nltk 

In [34]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/angelesmarin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [46]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelesmarin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
nltk.download('wordnet') #req for lem

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/angelesmarin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
path = ('/Users/angelesmarin/Desktop/CVE.csv')
df = pd.read_csv(path, encoding='ISO-8859-1', low_memory=False)
#was getting an error trying to decode using utf-8; got warrning to set low_memory=False 

### dropping columns

In [26]:
df = df.drop(['Phase', 'Votes', 'Comments'], axis=1)
#dropped outdated/ legacy fields

#### References has numbers, so just making the whole thing a string

In [27]:
df['References'] = df['References'].astype(str)

In [28]:
#df_columns = df.shape[1]  #columns
#print(f"num of columns: {df_columns}")  

Number of columns in the dataframe: 4


### check for null values 

In [67]:
print(df.isnull().sum())


Name                        0
Status                      0
Description                 0
References                  0
Tokenized_Description       0
Tokenized_Reference         0
Description_No_Stopwords    0
Lemmatize_Description       0
dtype: int64


### tokenization 

In [ ]:
# descriptions 

In [37]:
def tokenize_description(desc):
    return word_tokenize(desc)
df['Tokenized_Description'] = df['Description'].apply(tokenize_description)

In [39]:
print(df[['Description', 'Tokenized_Description']].head(3))


                                         Description  \
0  ip_input.c in BSD-derived TCP/IP implementatio...   
1  Buffer overflow in NFS mountd gives root acces...   
2  Execute commands as root via buffer overflow i...   

                               Tokenized_Description  
0  [ip_input.c, in, BSD-derived, TCP/IP, implemen...  
1  [Buffer, overflow, in, NFS, mountd, gives, roo...  
2  [Execute, commands, as, root, via, buffer, ove...  


In [ ]:
# references 

In [42]:
def tokenize_reference(ref):
    references = ref.split('|') #delimeter '|'
    tokenized_references = [r.split('::') for r in references] #delimeter ':'
    return tokenized_references
df['Tokenized_Reference'] = df['References'].apply(tokenize_reference)

In [44]:
print(df[['References', 'Tokenized_Reference']].head(3))

                                          References  \
0  BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...   
1  BID:121   |   URL:http://www.securityfocus.com...   
2  BID:122   |   URL:http://www.securityfocus.com...   

                                 Tokenized_Reference  
0  [[BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 ...  
1  [[BID:121   ], [   URL:http://www.securityfocu...  
2  [[BID:122   ], [   URL:http://www.securityfocu...  


### remove stopwords

In [ ]:
#descriptions

In [47]:
stop_words = set(stopwords.words('english'))

In [50]:
def remove_stopwords(tokenized_desc):
    return [word for word in tokenized_desc if word.lower() not in stop_words]
df['Description_No_Stopwords'] = df['Tokenized_Description'].apply(remove_stopwords)

In [51]:
print(df[['Description_No_Stopwords']].head(3))

                            Description_No_Stopwords
0  [ip_input.c, BSD-derived, TCP/IP, implementati...
1  [Buffer, overflow, NFS, mountd, gives, root, a...
2  [Execute, commands, root, via, buffer, overflo...


### lemmatization

In [ ]:
#descriptions 

In [56]:
lemmatization = WordNetLemmatizer()
def lemmatize_words(tokenized_desc):
    return [lemmatization.lemmatize(word) for word in tokenized_desc]
df['Lemmatize_Description'] = df['Description_No_Stopwords'].apply(lemmatize_words)

In [57]:
print(df[['Lemmatize_Description']].head(3))


                               Lemmatize_Description
0  [ip_input.c, BSD-derived, TCP/IP, implementati...
1  [Buffer, overflow, NFS, mountd, give, root, ac...
2  [Execute, command, root, via, buffer, overflow...


### output dataframe 

In [60]:
output_dataframe = df[['Name', 'Status', 'Lemmatize_Description', 'Tokenized_Reference']]

In [65]:
print(df.tail())

                 Name     Status  \
324604  CVE-2024-6320  Candidate   
324605  CVE-2024-6321  Candidate   
324606  CVE-2024-6322  Candidate   
324607  CVE-2024-6323  Candidate   
324608  CVE-2024-6324  Candidate   

                                              Description References  \
324604  ** RESERVED ** This candidate has been reserve...        nan   
324605  ** RESERVED ** This candidate has been reserve...        nan   
324606  ** RESERVED ** This candidate has been reserve...        nan   
324607  ** RESERVED ** This candidate has been reserve...        nan   
324608  ** RESERVED ** This candidate has been reserve...        nan   

                                    Tokenized_Description Tokenized_Reference  \
324604  [*, *, RESERVED, *, *, This, candidate, has, b...             [[nan]]   
324605  [*, *, RESERVED, *, *, This, candidate, has, b...             [[nan]]   
324606  [*, *, RESERVED, *, *, This, candidate, has, b...             [[nan]]   
324607  [*, *, RESERVED, *

In [66]:
output_dataframe.to_csv('/Users/angelesmarin/Desktop/output_dataframe.csv', index=False)
